In [78]:
def GenerateMatrix (Moore_VCF, Clone_no, **kwargs):
    import pandas as pd

    SigProfMatrix = pd.DataFrame (index = range ( len (Moore_VCF)),  columns = ["Project", "Sample", "ID", "Genome", "mut_type", "chrom",	"pos_start","pos_end",	"ref",	"alt"	,"Type"]) 
    SigProfMatrix["Project"] = "BioData"
    SigProfMatrix["Sample"] = str(Clone_no)
    SigProfMatrix["ID"] = Moore_VCF["pos"]
    SigProfMatrix["Genome"] = "GRCh37"
    SigProfMatrix["mut_type"] = "SNP"
    SigProfMatrix["chrom"] = Moore_VCF["chr"]
    SigProfMatrix["pos_start"] = Moore_VCF["start"]
    SigProfMatrix["pos_end"] = Moore_VCF["end"]
    SigProfMatrix["ref"] = Moore_VCF["ref"]
    SigProfMatrix["alt"] = Moore_VCF["alt"]
    SigProfMatrix["Type"] = "SOMATIC"

    pd.DataFrame(SigProfMatrix).to_csv( kwargs["OUTPUT_PATH"], index=False, header = True,  sep="\t")


def extract (Moore_VCF, **kwargs):
    import numpy as np

    # Clone number 대로  mutation_id를 뽑아주기
    with open( kwargs ["DEICISION_MEMBERSHIP_PATH"], "r") as input_file:
        membership = np.array( [int(line.strip()) for line in input_file] )
        npvaf = pd.read_csv ( kwargs ["NPVAF_PATH"], sep = "\t")
        for Clone_no in sorted(np.unique ( membership) ) :
            npvaf_Clone_no =  (npvaf.iloc [np.where(membership == Clone_no)[0] ])
            pos_Clone_no =  list ( npvaf_Clone_no.iloc[:, 0] )
            print (pos_Clone_no)

            Moore_VCF_Clone_no = Moore_VCF[Moore_VCF['pos'].isin( pos_Clone_no ) == True]
            
            kwargs["OUTPUT_PATH"] = kwargs["OUTPUT_DIR"] + "/clone{}.txt"
            GenerateMatrix (Moore_VCF_Clone_no, Clone_no, **kwargs)

    # whole
    kwargs["OUTPUT_PATH"] = kwargs["OUTPUT_DIR"] + "/whole.txt"
    GenerateMatrix (Moore_VCF, "whole", **kwargs)


if __name__ == "__main__":
    from SigProfilerMatrixGenerator.scripts import SigProfilerMatrixGeneratorFunc as matGen
    import pandas as pd
    import argparse

    Moore_VCF = pd.read_csv("/data/project/Alzheimer/CLEMENT/resource/paper/whole_info.txt", sep = "\t")

    # column을 소문자로 바꾸기
    Moore_VCF.rename(columns=lambda x: x.lower(), inplace=True)

    # lexicographically 정렬하기
    custom_order = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "X", "Y"]
    Moore_VCF['chr_categorical'] = pd.Categorical(Moore_VCF['chr'], categories=custom_order, ordered=True)
    Moore_VCF = Moore_VCF.sort_values(by= ['chr_categorical', 'start']).reset_index(drop=True).drop ("chr_categorical", axis = 1)

    # 앞에 "chr" 붙여주기
    Moore_VCF["pos"]  = "chr" + Moore_VCF["chr"] + "_" + Moore_VCF["start"].astype("str")


    kwargs = {}
    kwargs ["DEICISION_MEMBERSHIP_PATH"] = "/data/project/Alzheimer/CLEMENT/03.combinedoutput/3.BioData/Moore_1D/adrenal_gland_zona_glomerulosa/PD28690-L1/CLEMENT_hard_1st.membership.txt"
    kwargs ["NPVAF_PATH"] = "/data/project/Alzheimer/CLEMENT/02.npvaf/3.BioData/Moore_1D/adrenal_gland_zona_glomerulosa/PD28690-L1/npvaf.txt"


    extract (Moore_VCF, **kwargs)

['chr1_22106810', 'chr1_43264031', 'chr1_44056898', 'chr1_45021128', 'chr1_46657666', 'chr1_82077397', 'chr1_104117749', 'chr1_104890064', 'chr1_107245692', 'chr1_108544745', 'chr1_119479146', 'chr1_218301305', 'chr10_1601169', 'chr10_18938027', 'chr10_106975869', 'chr11_12064417', 'chr11_49401108', 'chr11_74754667', 'chr11_82118701', 'chr11_83420692', 'chr11_112911290', 'chr11_122451708', 'chr12_53184439', 'chr12_58814861', 'chr12_101951176', 'chr12_116340431', 'chr12_119248872', 'chr13_28858579', 'chr13_75117852', 'chr13_79435091', 'chr13_90599888', 'chr13_98535008', 'chr13_101188881', 'chr14_25799336', 'chr14_101732472', 'chr14_106479195', 'chr15_53107520', 'chr16_1760524', 'chr16_26901264', 'chr16_58888931', 'chr16_66430959', 'chr16_74942782', 'chr17_11873996', 'chr17_40279859', 'chr17_45368707', 'chr18_13802451', 'chr18_22123172', 'chr18_50032633', 'chr19_28305475', 'chr19_31465721', 'chr19_31747121', 'chr19_52206820', 'chr2_13453480', 'chr2_35283717', 'chr2_52327241', 'chr2_52822

In [59]:
from SigProfilerMatrixGenerator.scripts import SigProfilerMatrixGeneratorFunc as matGen
import pandas as pd

Moore_VCF = pd.read_csv("/data/project/Alzheimer/CLEMENT/resource/paper/whole_info.txt", sep = "\t")

# column을 소문자로 바꾸기
Moore_VCF.rename(columns=lambda x: x.lower(), inplace=True)

# lexicographically 정렬하기
custom_order = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "X", "Y"]
Moore_VCF['chr_categorical'] = pd.Categorical(Moore_VCF['chr'], categories=custom_order, ordered=True)
Moore_VCF = Moore_VCF.sort_values(by= ['chr_categorical', 'start']).reset_index(drop=True).drop ("chr_categorical", axis = 1)

# 앞에 "chr" 붙여주기
Moore_VCF["pos"]  = "chr" + Moore_VCF["chr"] + "_" + Moore_VCF["start"].astype("str")

In [60]:
# Filter rows where 'pos' is in the list 'a'
list_of_interest = ["chr1_115672", "chr1_525383", "chr1_540719"]
Moore_VCF = Moore_VCF[Moore_VCF['pos'].isin( list_of_interest ) == True]
Moore_VCF

,chr,start,end,ref,alt,depth_count,alt_count,vaf,sample,donorid,tissue,func,gene,pos
0,1,115672,115672,C,T,13,8,0.615385,PD43850u_P50_RTM_B10,PD43850,colon_crypt,intergenic,"OR4F5,LOC729737",chr1_115672
1,1,525383,525383,G,A,5,5,1.000000,PD28690bw_APP_3_D5,PD28690,appendix_crypt,intergenic,"OR4F3,OR4F29",chr1_525383
2,1,540719,540719,C,G,15,11,0.733333,PD43851t_P52_PRST_F11,PD42565,prostate_acinus,intergenic,"OR4F3,OR4F29",chr1_540719


In [61]:
SigProfMatrix = pd.DataFrame (index = range ( len (Moore_VCF)),  columns = ["Project", "Sample", "ID", "Genome", "mut_type", "chrom",	"pos_start","pos_end",	"ref",	"alt"	,"Type"]) 
SigProfMatrix["Project"] = "BioData"
SigProfMatrix["Sample"] = "clone1"
SigProfMatrix["ID"] = Moore_VCF["pos"]
SigProfMatrix["Genome"] = "GRCh37"
SigProfMatrix["mut_type"] = "SNP"
SigProfMatrix["chrom"] = Moore_VCF["chr"]
SigProfMatrix["pos_start"] = Moore_VCF["start"]
SigProfMatrix["pos_end"] = Moore_VCF["end"]
SigProfMatrix["ref"] = Moore_VCF["ref"]
SigProfMatrix["alt"] = Moore_VCF["alt"]
SigProfMatrix["Type"] = "SOMATIC"

SigProfMatrix

,Project,Sample,ID,Genome,mut_type,chrom,pos_start,pos_end,ref,alt,Type
0,BioData,clone1,chr1_115672,GRCh37,SNP,1,115672,115672,C,T,SOMATIC
1,BioData,clone1,chr1_525383,GRCh37,SNP,1,525383,525383,G,A,SOMATIC
2,BioData,clone1,chr1_540719,GRCh37,SNP,1,540719,540719,C,G,SOMATIC
